# Multisensory integration architecture for sign-language recognition
## Project for Neuro-Inspired Systems Engineering course at TUM

Authors of the project: Tatyana Klimenko and Cristina Gil

This script was written by Cristina Gil

# Part 1: modeling sign-language gestures

The following code aims to model from a basic perspective sign-language gestures of american sign language.
<img src="american_sign_language.png">


In [1]:
import numpy as np
import pandas as pd
from collections import *

In [2]:
# Letters
letters = {}

# Fingers: Index, Middle, Ring, Little
fingers = ['index','middle','ring','little']

# Joints
'''
Angles for fingers: 
    - MCP_fe: metacarpophalangeal flexion/extension 
    - MCP_aa: metacarpophalangeal abduction/adduction
    - PIP: Proximal-interphalangeal 

Angles for thumb:
    - TMC_fe: trapeziometacarpal flexion/extension
    - TMC_aa: trapeziometacarpal abduction/adduction
    - MCP_fe: metacarpophalangeal flexion/extension
'''
angles = {'MCP_fe','MCP_aa','PIP'}
angles_thumb = {'TMC_fe','TMC_aa','MCP_fe'}


# Initialize angles to 0 (Rest position of the hand)
default_params = defaultdict(dict)

for finger in fingers:
    for angle in angles:
        default_params[finger][angle] = 0
# Thumb
for angle in angles_thumb:
    default_params['thumb'][angle]=0

# Create a dataframe from default_parameters
df = pd.DataFrame.from_dict(default_params)
print(df)

# Transform the dataframe to a row vector
array = df.as_matrix().ravel()
array = array[~np.isnan(array)]

# Number of parameters
n_params = len(array)

        index  little  middle  ring  thumb
MCP_aa    0.0     0.0     0.0   0.0    NaN
MCP_fe    0.0     0.0     0.0   0.0    0.0
PIP       0.0     0.0     0.0   0.0    NaN
TMC_aa    NaN     NaN     NaN   NaN    0.0
TMC_fe    NaN     NaN     NaN   NaN    0.0


In [3]:
# Parametrize the letters

# Letter A
A = df.copy()
A.loc['MCP_fe':'PIP','index':'ring']=90
A.loc['TMC_aa','thumb']=-45
letters['A']=A

# Letter B
B = df.copy()
B.loc['TMC_fe','thumb']=180
letters['B']=B

# Letter D
D = df.copy()
D.loc['MCP_fe','little':'thumb']=45
D.loc['PIP','little':'ring']=60
D.loc['TMC_fe','thumb'] = 90
letters['D']=D

# Letter E
E = df.copy()
E.loc['MCP_fe','index':'thumb']=90
E.loc['PIP','index':'ring']=90
E.loc['TMC_aa','thumb']=-90
letters['E']=E

# Letter F
F = df.copy()
F.loc['MCP_fe','index']=30
F.loc['PIP','index']=90
F.loc['TMC_fe','thumb']=45
F.loc['MCP_fe','thumb']=90
letters['F']=F


In [4]:
# Print letters structure for checking
for key, value in letters.items():
    print(key, value)


A         index  little  middle  ring  thumb
MCP_aa    0.0     0.0     0.0   0.0    NaN
MCP_fe   90.0    90.0    90.0  90.0    0.0
PIP      90.0    90.0    90.0  90.0    NaN
TMC_aa    NaN     NaN     NaN   NaN  -45.0
TMC_fe    NaN     NaN     NaN   NaN    0.0
B         index  little  middle  ring  thumb
MCP_aa    0.0     0.0     0.0   0.0    NaN
MCP_fe    0.0     0.0     0.0   0.0    0.0
PIP       0.0     0.0     0.0   0.0    NaN
TMC_aa    NaN     NaN     NaN   NaN    0.0
TMC_fe    NaN     NaN     NaN   NaN  180.0
D         index  little  middle  ring  thumb
MCP_aa    0.0     0.0     0.0   0.0    NaN
MCP_fe    0.0    45.0    45.0  45.0   45.0
PIP       0.0    60.0    60.0  60.0    NaN
TMC_aa    NaN     NaN     NaN   NaN    0.0
TMC_fe    NaN     NaN     NaN   NaN   90.0
E         index  little  middle  ring  thumb
MCP_aa    0.0     0.0     0.0   0.0    NaN
MCP_fe   90.0    90.0    90.0  90.0   90.0
PIP      90.0    90.0    90.0  90.0    NaN
TMC_aa    NaN     NaN     NaN   NaN  -90.0
TMC

In [5]:
# Function for creatig samples
def create_samples(letter,n_samples,variance):
    # Convert to array and eliminate nan values
    array = letter.as_matrix().ravel()
    array = array[~np.isnan(array)]
    
    # Create samples and add gausian noise
    data = np.tile(array, (n_samples,1))
    noise = np.random.normal(0, variance, data.shape)
    params = data+noise
    
    return params

In [47]:
# Initialize variables
n_samples = 1000
variance = 5 #For noise: 5 degrees in all the angles  
n_letters = len(letters)

# Create the parameters structure to feed the neural network
params = np.zeros((n_letters*n_samples,n_params))
labels = np.zeros(n_letters*n_samples)
key_labels = {} # Dictionary to store the number assigned to each letter

l=0
for key, letter in letters.items():
    
    # Params
    new_params = create_samples(letter,n_samples,variance)
    params[(l*n_samples):(l*n_samples+n_samples),:] = new_params    
    l+=1
    
    # Labels
    labels[((l-1)*n_samples):((l-1)*n_samples+n_samples)]=l
    key_labels[key]=l

# Part 2: Setting the NN

In [36]:
import torch
import torch.nn as nn

## 0. Prepare the data

In [84]:
# Split the data in training set, validation set and test set
# Input data stored in params, labels stored in labels

# Create random list of intergers to mask the input data
a = np.arange(4999)
np.random.shuffle(a)


# Test set
mask_test = np.zeros(n_letters*n_samples, dtype=bool)
trues_test = a[:100]
mask_test[trues_test] = True

test_data = params[mask_test,:]
test_labels = labels[mask_test]

# Validation set
mask_val = np.zeros(n_letters*n_samples, dtype=bool)
trues_val = a[100:200]
mask_val[trues_val]=True

val_data = params[mask_val,:]
val_labels = labels[mask_val]

# Train set
mask_train = (~mask_test)*(~mask_val)
train_data = params[mask_train,:]
train_labels = labels[mask_train]


print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))

Train size: 4800
Val size: 100
Test size: 100


## 1. Define the network

In [ ]:
class twoLayerNet(nn.Module):
    
    def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.
    """
    super(TwoLayerNet, self).__init__()
    self.linear1 = nn.Linear(D_in, H)
    self.linear2 = nn.Linear(H, D_out)
    
    def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary (differentiable) operations on Tensors.
    """
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred       

## 2. Define a loss function and an optimizer

In [ ]:
# Construct our loss function and an Optimizer
loss_fn = nn.MSELoss(reduction='elementwise_mean') # reduction = 'sum'
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [34]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, train_data.shape[0], 100, train_data.shape[1]

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)


# Construct our model by instantiating the class defined above.
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer
loss_fn = nn.MSELoss(reduction='sum') # reduction = 'elementwise_mean'
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = loss_fn(y_pred, y)
  print(t, loss.item())

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
